<a href="https://colab.research.google.com/github/JangHanjun/Natural_Language_Programming/blob/main/Chapter11/WordRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
vocab = list(set(sentence))
print(vocab)

['memory', 'Repeat', 'the', 'for', 'best', 'medicine', 'is']


In [5]:
word2index = {tkn : i for i, tkn in  enumerate(vocab, 1)}
word2index['<unk>'] = 0
print(word2index)

{'memory': 1, 'Repeat': 2, 'the': 3, 'for': 4, 'best': 5, 'medicine': 6, 'is': 7, '<unk>': 0}


In [7]:
idx2word = {tkn : i for i, tkn in word2index.items()}
print(idx2word)

{1: 'memory', 2: 'Repeat', 3: 'the', 4: 'for', 5: 'best', 6: 'medicine', 7: 'is', 0: '<unk>'}


In [8]:
def build_data(sentence, word2index):
  encoded = [word2index[token] for token in sentence] # 각 문자를 정수로
  input_seq, label_seq = encoded[:-1], encoded[1:] # 입력, 레이블 시퀀스 분리

  input_seq = torch.LongTensor(input_seq).unsqueeze(0)
  label_seq = torch.LongTensor(label_seq).unsqueeze(0)

  return input_seq, label_seq

In [10]:
X, Y = build_data(sentence, word2index)
print(X)
print(Y)

tensor([[2, 7, 3, 5, 6, 4]])
tensor([[7, 3, 5, 6, 4, 1]])


In [12]:
class Net(nn.Module):
  def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
    super(Net, self).__init__()
    self.embedding_layer = nn.Embedding(num_embeddings=vocab_size,
                                        embedding_dim=input_size)
    self.rnn_layer = nn.RNN(input_size, hidden_size, 
                            batch_first=batch_first)
    self.linear = nn.Linear(hidden_size, vocab_size)
  
  def forward(self, x):
    # 1 - 임베딩 층
    # 크기변화 : (배치 크기, 시퀸스 길이) => (배치 크기, 시퀸스 길이, 임베딩 차원)
    output = self.embedding_layer(x)

    # 2 - RNN층
    # 크기변화 : (배치 크기, 시퀸스 길이, 임베딩 차원)
    # => output : (배치 크기, 시퀸스 길이, 은닉층 차원)
    # hidden : (1, 배치 크기, 은닉층 크기)
    output, hidden = self.rnn_layer(output)

    # 3 - 최종 출력층
    # 크기 변화 : (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
    output = self.linear(output)

    # 4 - view를 통해서 배치 차원 제거
    # 크기 변화 : (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기,*시퀸스 길이, 단어장 크기)
    return output.view(-1, output.size(2))

In [13]:
# 하이퍼 파라미터
vocab_size = len(word2index)
input_size = 5    # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [15]:
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters())

In [16]:
output = model(X)
print(output)

tensor([[ 0.1653, -0.1343,  0.3411, -0.1654, -0.0481,  0.2472, -0.0657, -0.3647],
        [ 0.2336,  0.5395,  0.5128, -0.1267, -0.2807,  0.1619, -0.1223, -0.5432],
        [-0.0473, -0.7080, -0.1636,  0.0229, -0.2342,  0.3216,  0.0451, -0.0826],
        [ 0.1025, -0.4546, -0.0592, -0.0205, -0.0484,  0.1557, -0.0595, -0.3603],
        [ 0.0747, -0.2633, -0.1121, -0.1524,  0.0469,  0.1235, -0.0709, -0.2964],
        [ 0.1439,  0.1084,  0.4935,  0.0635, -0.5253,  0.1524, -0.5706, -0.2625]],
       grad_fn=<ViewBackward0>)


In [18]:
print(output.shape)

torch.Size([6, 8])


In [19]:
decode = lambda y : [idx2word.get(x) for x in y]

In [21]:
for step in range(201):
  # 경사 초기화
  optimizer.zero_grad()
  # 순방향 전파
  output = model(X)
  # 손실값 계산
  loss = loss_function(output, Y.view(-1))
  # 역방향 전파
  loss.backward()
  #매개변수 업데이트
  optimizer.step()

  if step % 40 == 0 :
    print("[{:02d}/201] {:.4f} ".format(step+1, loss))
    pred = output.softmax(-1).argmax(-1).tolist()
    print(" ".join(["Repeat"] + decode(pred)))
    print()

[01/201] 0.1499 
Repeat is the best medicine for memory

[41/201] 0.0977 
Repeat is the best medicine for memory

[81/201] 0.0699 
Repeat is the best medicine for memory

[121/201] 0.0533 
Repeat is the best medicine for memory

[161/201] 0.0424 
Repeat is the best medicine for memory

[201/201] 0.0347 
Repeat is the best medicine for memory

